In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta
from tqdm import tqdm
import util_fun as uf
from enctoken import get_kite
kite = get_kite()
import sqlite3
from swifter import set_defaults
set_defaults(
    progress_bar=False,

)
import warnings
warnings.filterwarnings("ignore")

In [3]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

i=1
error_list = []
# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
# inst_filter.query("Symbol == 'HDFCBANK'")

nifty_500 = pd.read_csv('../data/ind_nifty500list.csv')
nifty_500 = nifty_500.merge(inst_filter, on = 'Symbol')
inst_dict = dict(zip(nifty_500.Symbol, nifty_500.instrument_token))


In [4]:
def get_day_data(kite, instrument, time_frame, start_date,end_date):
    results = pd.DataFrame()
    df = pd.DataFrame(kite.historical_data(instrument,start_date, end_date,time_frame))
    results = results.append(df) 
    return results

def get_data_parllel(kite, instrument, time_frame, start_date, end_date):
    df_final = pd.DataFrame()
    temp_sd = pd.to_datetime(start_date).strftime('%Y-%m-01')
    if (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days > 28:
        df = pd.DataFrame(pd.date_range(temp_sd ,end_date,freq="MS")).rename(columns={0:'start_date'})
        df['end_date'] = df.shift(-1)
        df['end_date'].fillna(pd.to_datetime(end_date), inplace=True)
    else:
        df = pd.DataFrame({"start_date": pd.to_datetime(start_date), "end_date": pd.to_datetime(end_date)}, index=[0])
    
    df['tf'] = time_frame
    df['inst'] = instrument
    df['fetch_data'] = df.swifter.apply(lambda x: pd.DataFrame(kite.historical_data(x['inst'], x['start_date'], x['end_date'], x['tf'])),axis = 1)
    for i in df['fetch_data']:
        df_final = df_final.append(i)
    df_final.rename(columns = {'date': "Date",'close':'Close', 'high':'High', 'low':"Low", 'open': "Open","volume":"Volume"}, inplace = True)
    df_final= df_final[(pd.to_datetime(df_final['Date'].dt.date) >= pd.to_datetime(start_date)) &  \
                      (pd.to_datetime(df_final['Date'].dt.date) <= pd.to_datetime(end_date))]
    df_final.set_index('Date', inplace=True)
    return df_final

## data pull

In [5]:
today_date = '2023-05-26'
start_dt = '2023-05-26'
end_dt = '2023-05-27'
time_frame = '15minute'

# import sql 
# conn = sql.create_connection('../data/Nifty_500.db')
# cursor_obj = conn.cursor()
# sql.delete_table(conn,'fivteen_minute')
df_all = pd.DataFrame()
for symbol, instument in tqdm(inst_dict.items()):
    if i:
        # print(symbol, instument)
        try:
            df_all = df_all.append(get_data_parllel(kite, instument, time_frame , start_dt,end_dt).assign(stock = symbol))
            # df_day['stock'] = symbol
            # df_all = df_all.append(df_day)
            # df_all.reset_index(drop = True)
            # df_day.to_sql('fivteen_minute',conn, if_exists="append",index = True, index_label = 'Date')
            
                # directory = f'../data/historical/{time_frame}/{symbol}'
                # if not os.path.exists(directory):
                #     os.makedirs(directory)
                # df_day.to_parquet(f'{directory}/part0.parquet')
        except:
                print(instument)
                error_list.append(instument)

    # break

    i += 1
print(error_list)


100%|██████████| 497/497 [02:24<00:00,  3.45it/s]

[]


In [6]:
# df_all = sql.sql_to_pd(conn,"fivteen_minute" )
df_all = df_all.reset_index()
df_all['Date']  = pd.to_datetime(df_all['Date']).dt.tz_localize(None)
df_all['date_only'] = pd.to_datetime(df_all['Date'].dt.date)

In [7]:
df_all.head()

,Date,Open,High,Low,Close,Volume,stock,date_only
0,2023-05-26 09:00:00,23962.00,23962.00,23962.00,23962.00,0,3MINDIA,2023-05-26
1,2023-05-26 09:15:00,23962.00,24079.10,23952.75,24055.90,181,3MINDIA,2023-05-26
2,2023-05-26 09:30:00,24055.90,24055.90,23910.30,23925.00,119,3MINDIA,2023-05-26
3,2023-05-26 09:45:00,23925.00,23966.35,23823.85,23869.45,67,3MINDIA,2023-05-26
4,2023-05-26 10:00:00,23869.45,23933.15,23869.45,23924.90,32,3MINDIA,2023-05-26


## inside candle 

In [9]:
import talib
candle_names = talib.get_function_groups()['Pattern Recognition']

In [10]:
# first candle open to high change is 1%
# second candle is .1% and on the top of candle 

In [20]:
def inside_candle(df,first_candle_size , second_candle_size):
    df['per_chg'] =(df['Open'] - df['Close'])/ df['Open']
    print(df)
    df.loc[df.Date.dt.time == pd.to_datetime('09:15:00').time(),'High'].unique()[0] 

In [7]:
# df_all.query('stock == "RELIANCE"').groupby("stock").apply(lambda x: inside_candle(x, .01,.001))

In [23]:
# extract OHLC 
op = df_all['Open']
hi = df_all['High']
lo = df_all['Low']
cl = df_all['Close']
# create columns for each pattern
for candle in ['CDLCLOSINGMARUBOZU',"CDLMARUBOZU"]:
    print(candle)
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df_all[candle] = getattr(talib, candle)(op, hi, lo, cl)

CDLCLOSINGMARUBOZU
CDLMARUBOZU


In [30]:
df_all.CDLCLOSINGMARUBOZU.value_counts()

 0      13580
-100      874
 100      765
Name: CDLCLOSINGMARUBOZU, dtype: int64

In [38]:
df_all[(df_all.Date.dt.time == pd.to_datetime('09:15:00').time()) & (df_all.CDLMARUBOZU != 0)].stock.unique()

array(['ABB', 'BHARTIARTL', 'SFL', 'TCNSBRANDS', 'NIACL'], dtype=object)

## 6 candle breakdown

In [8]:
def first_six_candle_setup(df):
    f_high = df.loc[df.Date.dt.time == pd.to_datetime('09:15:00').time(),'High'].unique()[0] 
    f_low = df.loc[df.Date.dt.time == pd.to_datetime('09:15:00').time(),'Low'].unique()[0]
    df['f_high'] = f_high
    df["f_low"] = f_low
    df['per_chg'] = (abs(f_high - f_low)/ f_low) * 100
    df['break_f_high'] = np.where(df['High'] > df['f_high'], "brk", 'not_b')
    df['break_f_low'] = np.where(df['Low'] < df['f_low'],"brk", 'not_b')
    df['break_any_side'] = np.where((df['break_f_high'] == 'not_b') & (df['break_f_low'] == 'not_b'),'not_b',"brk") 
    # print(df)
    # return df
    first_five_candle = df.loc[df.Date.dt.time <= pd.to_datetime('10:30:00').time(),'break_any_side'].unique()
    for i in first_five_candle:
        if 'brk' in i:
            return [False,df['per_chg'].unique()[0]]
    return [True,df['per_chg'].unique()[0]]


In [13]:
# df_wipro = df_all[(df_all.date_only == '2023-05-05') & (df_all.stock == 'CGCL')]
# df_filter_w = df_wipro.groupby(['date_only','stock']).apply(first_six_candle_setup).reset_index().rename(columns={0:"flag"})

In [9]:
df_filter = df_all.groupby(['date_only','stock']).apply(first_six_candle_setup).reset_index().rename(columns={0:"flag"})
df_filter = pd.concat([df_filter.drop('flag', axis= 1),df_filter.flag.apply(pd.Series).rename(columns = {0 :'flag', 1: 'per_chg'})],axis = 1)

In [10]:
# sort by first candle minimum % chg
df_selected = df_filter[(df_filter.flag) & (df_filter.date_only == today_date)].sort_values('per_chg')#.head(50)
df_first_candle = df_all.loc[df_all.Date.dt.time == pd.to_datetime('09:15:00').time(),:]
df_selected = df_selected.merge(df_first_candle, how  = 'left', on = ['date_only','stock'])
df_selected['ltp_cols'] = "NSE" + ":" +df_selected['stock']
df_selected = df_selected[df_selected.Open.between(100,2000)]

In [12]:
import time
import datetime
import xlwings as xw
wb=xw.Book("../data/five_candle.xlsx")
ws = wb.sheets["Sheet1"]


In [15]:

alrady_break = []
i = 0 
while(True):
    df_selected['ltp'] = df_selected['ltp_cols'].map(kite.ltp(df_selected.ltp_cols.unique().tolist())).apply(lambda x: x['last_price'])
    
    # if i == 0:
    #     print(i)
    #     i +=1
    #     df_selected_1 = df_selected[~((df_selected['ltp'] < df_selected['High']) &  (df_selected['ltp'] > df_selected['Low']))]
    #     df_selected_1['break_time'] = datetime.datetime.now().time()
    #     alrady_break.append(df_selected_1.stock.unique().tolist())

    df_selected_up = df_selected[(df_selected['ltp'] > df_selected['High'])]
    df_selected_down  =  df_selected[(df_selected['ltp'] < df_selected['Low'])]

    # df_selected_2 = df_selected_2.merge(df_selected_1[['date_only','stock','break_time']], how = 'left', on = ['date_only','stock'])
    # df_selected_2['break_time'] = np.where(df_selected_2.stock.isin(alrady_break),df_selected_2['break_time'],datetime.datetime.now().time() )
    # alrady_break.append(df_selected_1.stock.unique().tolist())
    # df_selected_2['break_time'] = df_selected_2['break_time'].astype(str)
    print(df_selected_up.stock.unique().tolist())
    print(df_selected_down.stock.unique().tolist())

    ws.clear_contents()
    ws.range("A1").options(index=False).value = df_selected_up
    ws.range(f"A{df_selected_up.shape[0]+3}").options(index=False).value = df_selected_down
    time.sleep(30)
    # break


['AXISBANK', 'TATASTEEL', 'TATACOMM', 'KPRMILL', 'GRAPHITE', 'BATAINDIA', 'COLPAL', 'TATAMOTORS', 'DELHIVERY', 'CHEMPLASTS', 'CUB', 'MFSL', 'TANLA', 'JUBLPHARMA', 'QUESS', 'CGPOWER', 'MARICO', 'ORIENTELEC', 'RADICO']
['HGS', 'HINDZINC', 'PAYTM', 'APLAPOLLO', 'GUJALKALI', 'JSWENERGY', 'JKLAKSHMI', 'APLLTD', 'RITES']
['AXISBANK', 'POLYMED', 'TATASTEEL', 'TATACOMM', 'KPRMILL', 'GRAPHITE', 'BATAINDIA', 'COLPAL', 'TATAMOTORS', 'DELHIVERY', 'CHEMPLASTS', 'CUB', 'MFSL', 'TANLA', 'QUESS', 'CGPOWER', 'MARICO', 'ORIENTELEC', 'RADICO']
['HGS', 'HINDZINC', 'PAYTM', 'APLAPOLLO', 'GUJALKALI', 'JSWENERGY', 'JKLAKSHMI', 'APLLTD', 'RITES']
['AXISBANK', 'TATASTEEL', 'TATACOMM', 'KPRMILL', 'GRAPHITE', 'BATAINDIA', 'COLPAL', 'TATAMOTORS', 'DELHIVERY', 'CHEMPLASTS', 'CUB', 'MFSL', 'TANLA', 'QUESS', 'CGPOWER', 'MARICO', 'ORIENTELEC', 'RADICO']
['HGS', 'HINDZINC', 'PAYTM', 'APLAPOLLO', 'GUJALKALI', 'JSWENERGY', 'JKLAKSHMI', 'APLLTD', 'FSL', 'RITES']
['AXISBANK', 'TATASTEEL', 'LATENTVIEW', 'TATACOMM', 'KPRMIL

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [21]:
df_selected['quantity'] =(45000/df_selected['ltp']) * 5

In [31]:
df_selected['near_High'] = df_selected['High'] - df_selected['High']  * .005

In [35]:
df_selected['Low'] + df_selected['Low']  * .001

1     528.97845
2     472.37190
3     165.46530
6     692.29160
7     123.12300
        ...    
89    240.29005
90    985.48450
91    118.61850
94    173.92375
96    459.45900
Name: Low, Length: 67, dtype: float64

In [34]:
df_selected

,date_only,stock,flag,per_chg,Date,Open,High,Low,Close,Volume,ltp_cols,ltp,quantity,near_High
1,2023-05-25,MARICO,True,0.501467,2023-05-25 09:15:00,530.65,531.10,528.45,530.15,63163,NSE:MARICO,534.55,420.914788,528.44450
2,2023-05-25,SUVENPHAR,True,0.550964,2023-05-25 09:15:00,471.90,474.50,471.90,472.00,3322,NSE:SUVENPHAR,474.00,474.683544,472.12750
3,2023-05-25,ONGC,True,0.574713,2023-05-25 09:15:00,166.25,166.25,165.30,165.45,454215,NSE:ONGC,163.05,1379.944802,165.41875
6,2023-05-25,HOMEFIRST,True,0.628976,2023-05-25 09:15:00,695.95,695.95,691.60,693.15,3352,NSE:HOMEFIRST,695.85,323.345549,692.47025
7,2023-05-25,FEDERALBNK,True,0.691057,2023-05-25 09:15:00,123.55,123.85,123.00,123.65,1219104,NSE:FEDERALBNK,124.15,1812.323802,123.23075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2023-05-25,GAEL,True,2.770256,2023-05-25 09:15:00,240.60,246.70,240.05,244.00,44240,NSE:GAEL,245.15,917.805425,245.46650
90,2023-05-25,KPITTECH,True,2.884713,2023-05-25 09:15:00,984.50,1012.90,984.50,1004.00,867024,NSE:KPITTECH,1006.00,223.658052,1007.83550
91,2023-05-25,NIACL,True,2.911392,2023-05-25 09:15:00,118.75,121.95,118.50,120.75,282899,NSE:NIACL,121.50,1851.851852,121.34025
94,2023-05-25,GICRE,True,4.748201,2023-05-25 09:15:00,174.60,182.00,173.75,179.50,665695,NSE:GICRE,181.60,1238.986784,181.09000


In [ ]:
uf.alert_msg()

## 4 consecutive candle 

In [20]:
df_all['per_chg'] = abs(df_all['Open'] - df_all['Close'])  / df_all['Close']

In [21]:
def four_candle_setup(df,per_chng = 0.0002):
    # 1. get % chg in open close 
    # 2. consecutive 4 true case 
    df['fst'] = df['per_chg'].shift(-1)
    df['scnd'] = df['per_chg'].shift(-2)
    df['third'] = df['per_chg'].shift(-3)

    # print(df)
    df['cond_flag'] = np.where(
        (df['fst'] <= per_chng) & 
        (df['scnd'] <= per_chng) & 
        (df['third'] <=per_chng)  & 
        (df['per_chg'] <= per_chng),
        True,
        False
        )
    # print(df)
    return df['cond_flag'].sum()

In [22]:
# df_ind = df_all[(df_all.stock == 'JSWENERGY') & (df_all.date_only == '2023-05-04')]
# df_four_ind = df_ind.groupby(['date_only','stock']).apply(four_candle_setup).reset_index().rename(columns={0:"flag"})

In [24]:
df_part_date = df_all[(df_all.date_only == '2023-05-25')]
df_four = df_part_date.groupby(['date_only','stock']).apply(four_candle_setup).reset_index().rename(columns={0:"flag"})

In [27]:
df_four[(df_four.flag == 1) & (df_four.date_only == '2023-05-25')].stock.unique()

array(['ASTRAZEN', 'CREDITACC', 'GRASIM', 'HATSUN', 'INDIANB', 'MARUTI',
       'PGHH', 'POLICYBZR'], dtype=object)

In [ ]:
# df_all[(df_all.stock == "JSWENERGY") & (df_all.date_only == '2023-05-05')].head(50)